In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
llm = ChatOpenAI(model="gpt-4o-mini")

llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='네, 잘 지냈습니다! 당신은 어떻게 지내고 있나요? 궁금한 점이 있으면 언제든지 말씀해 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 12, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C1npM2eMCrb0nMsmsHnpyV4qJ0B68', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--0309f800-2065-4ba6-b432-1f1fdc9ff266-0', usage_metadata={'input_tokens': 12, 'output_tokens': 30, 'total_tokens': 42, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time


In [3]:
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

In [4]:
from langchain_core.messages import SystemMessage

# (4) 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

# (5) llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# (6) 생성된 llm 답변 출력
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1r6c9d277aYAe2CER8M1jvhu', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"Busan"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-C1npNbKGtwKRgrCBjAkvNxcwHBtA7', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cf507534-b1d6-4bf1-9679-288fad26d66a-0', tool_ca

In [5]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # (7) tool_dict를 사용하여 도구 함수를 선택
    print(tool_call["args"]) # (8) 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # (9) 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': 'Busan'}
Asia/Seoul (Busan) 현재시각 2025-08-07 14:51:42 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1r6c9d277aYAe2CER8M1jvhu', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"Busan"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-C1npNbKGtwKRgrCBjAkvNxcwHBtA7', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cf507534-b1d6-4bf1-9679-288fad26d66a-0', tool_

In [6]:
llm_with_tools.invoke(messages)

AIMessage(content='부산은 지금 2025년 8월 7일 오후 2시 51분입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 192, 'total_tokens': 216, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-C1npOmUsNOCr15m6QOUw6YEDUqo1j', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--fb64c744-22a3-48c1-9c2c-d96ebf8fac1c-0', usage_metadata={'input_tokens': 192, 'output_tokens': 24, 'total_tokens': 216, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")


In [8]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """ 주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown() 

    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {"get_current_time": get_current_time, "get_yf_stock_history": get_yf_stock_history}

llm_with_tools = llm.bind_tools(tools)

In [9]:
messages.append(HumanMessage("테슬라는 한달 전에 비해 주가가 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_xrp3NO3pPpxQuPxuLJdo8gor', 'function': {'arguments': '{"stock_history_input":{"ticker":"TSLA","period":"1mo"}}', 'name': 'get_yf_stock_history'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 283, 'total_tokens': 310, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-C1npPd0OjOvMYxo2bbpMTNRcbnt8T', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--f86025f2-3cd6-4a01-927f-8e8057f102f7-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_xrp3NO3pPpxQuPxuLJdo8gor', 'type': 'tool_call'}] usage_metadata={'inp

In [10]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

{'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2025-07-07 00:00:00-04:00 | 291.37 | 296.15 | 288.77 |  293.94 | 1.31178e+08 |           0 |              0 |\n| 2025-07-08 00:00:00-04:00 | 297    | 304.05 | 294.35 |  297.81 | 1.03247e+08 |           0 |              0 |\n| 2025-07-09 00:00:00-04:00 | 297.55 | 300.15 | 293.55 |  295.88 | 7.55868e+07 |           0 |              0 |\n| 2025-07-10 00:00:00-04:00 | 300.05 | 310.48 | 300    |  309.87 | 1.04365e+08 |           0 |              0 |\n| 2025-07-11 00:00:00-04:00 | 307.89 | 314.09 | 305.65 |  313.51 | 7.92364e+07 |           0 |              0 |\n| 2025-07-14 00:00:00-04:00 | 317.73 | 322.6  | 312.67 |  316.9  | 7.80434e+07 |           0 |              0 |\n| 2025-07-15 00:00:00-04:0

In [11]:
llm_with_tools.invoke(messages)

AIMessage(content='한 달 전인 2025년 7월 7일에 테슬라(TSLA)의 종가는 293.94달러였고, 현재 날짜인 2025년 8월 6일에는 319.91달러입니다.\n\n따라서, 테슬라는 한 달 사이에 주가가 상승했습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 1697, 'total_tokens': 1770, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-C1npTwn3aDSxjmgUvUoxfwTEyliEs', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--755d6585-817b-4ea2-97f2-0cbe666174d5-0', usage_metadata={'input_tokens': 1697, 'output_tokens': 73, 'total_tokens': 1770, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 08-4 스트림 출력

In [12]:
for c in llm.stream([HumanMessage("잘 지냈어? 한국 사회의 문제점에 대해 이야기해줘.")]):
    print(c.content, end='|') 

|저|는| 잘| 지|내|고| 있습니다|!| 한국| 사회|의| 문제|점|에| 대해| 말씀|드|리|자|면|,| 여러| 가지|가| 있지만| 몇| 가지| 주요|한| 문제|를| 소개|할| 수| 있습니다|.

|1|.| **|청|년| 실|업|**|:| 경제| 구조|의| 변화|와| 고|용| 시장|의| 경쟁| 심|화|로| 인해| 청|년|층|의| 실|업|률|이| 높|아|지고| 있습니다|.| 많은| 젊|은|이|들이| 적|합|한| 일|자|리를| 찾|기| 힘|들|어|하고|,| 이는| 사회|적인| 불|만|을| 야|기|하고| 있습니다|.

|2|.| **|주|거| 문제|**|:| 특히| 대|도|시|에서|의| 주|거| 비용|이| 급|등|하면서| 많은| 사람들이| 적|정|한| 가격|의| 주|택|을| 찾|기| 어려|워|지고| 있습니다|.| 이|로| 인해| 주|거| 불|안|이| 커|지고|,| 청|년|층| 특히| 문제가| 심|각|합니다|.

|3|.| **|성| 차|별| 및| 성|폭|력|**|:| 성|별|에| 따른| 불|균|형|과| 성|폭|력| 사건|은| 여|전히| 큰| 사회|적| 문제|입니다|.| 여성|에| 대한| 차|별|이| 사회| 여러| 분야|에서| 나타|나|고| 있으며|,| 그|에| 대한| 대|처|가| 필요한| 상황|입니다|.

|4|.| **|고|령|화| 사회|**|:| 고|령|화|가| 진행|되|면서| 연|금|,| 의료|,| 복|지| 시스템|에| 대한| 부담|이| 증가|하고| 있습니다|.| 이는| 사회| 전|반|에| 걸|쳐| 다양한| 문제|를| 야|기|하고|,| 더| 많은| 정책|적| 해결|책|이| 요구|되고| 있습니다|.

|5|.| **|정|신| 건강| 문제|**|:| 스트|레스|와| 경쟁| 심|화|로| 정신| 건강| 문제가| 증가|하고| 있습니다|.| 특히| 젊|은| 세|대|에서| 우|울|증|과| 불|안|장|애|가| 증가|하고| 있으며|,| 이에| 대한| 사회|적| 인|식|과| 지원|이| 필요|합니다|.

|이| 외|에도| 한국| 사회|에는| 다양한| 문제|들이| 존재|하지만|,

In [13]:
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

response = llm_with_tools.stream(messages)

# 파편화된 tool_call 청크를 하나로 합치기 
is_first = True
for chunk in response:    
    print("chunk type: ", type(chunk))
    
    if is_first:
        is_first = False
        gathered = chunk
    else:
        gathered += chunk
    
    print("content: ", gathered.content, "tool_call_chunk", gathered.tool_calls)

messages.append(gathered)

chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_LHtr9iwG9g7dDWMZuJ56n1Be', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_LHtr9iwG9g7dDWMZuJ56n1Be', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_LHtr9iwG9g7dDWMZuJ56n1Be', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': ''}, 'id': 'call_LHtr9iwG9g7dDWMZuJ56n1Be', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': 'Asia'}, 'id': 'call_LHtr9iwG9g7dDWMZuJ56n1Be', 'type': 'tool_c

In [14]:
gathered

AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_LHtr9iwG9g7dDWMZuJ56n1Be', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'service_tier': 'default'}, id='run--fadceb3c-b21d-4acb-b286-dea8c3aa45ce', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_LHtr9iwG9g7dDWMZuJ56n1Be', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_LHtr9iwG9g7dDWMZuJ56n1Be', 'index': 0, 'type': 'tool_call_chunk'}])

In [15]:
for tool_call in gathered.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # tool_dict를 사용하여 도구 이름으로 도구 함수를 선택
    print(tool_call["args"]) # 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재시각 2025-08-07 14:52:01 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_LHtr9iwG9g7dDWMZuJ56n1Be', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'service_tier': 'default'}, id='run--fadceb3c-b21d-4acb-b286-dea8c3aa45ce', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_LHtr9iwG9g7dDWMZuJ56n1Be', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_LHtr9iwG9g7dDWMZuJ56n1Be', 'index': 0, 'type': 'tool_call_chunk'}]),
 ToolMessage(content='Asia/S

In [16]:
for c in llm_with_tools.stream(messages):
    print(c.content, end='|')

|부|산|은| 지금| |202|5|년| |8|월| |7|일| 오후| |2|시| |52|분|입니다|.||